NegaMax Implementation for MiniMax

In [ ]:
def _value(self, board: chess.Board, depth: int) -> int:
    if score := self._evaluate(board):
        return score
    if depth == 0:
        return self._absolute_heuristic(board)

    scores = []
    for move in board.legal_moves:
        board.push(move)
        scores.append(-1 * self._value(board, depth - 1))
        board.pop()

    return max(scores)


MiniMaxEngine._value = _value


def _evaluate_move(self, board: chess.Board, move: chess.Move, depth: int):
    board.push(move)
    score = self._value(board, depth)
    score *= self.PLAYER_MULTIPLIER[board.turn]
    board.pop()
    return ScoredMove(score=score, move=move)


MiniMaxEngine._evaluate_move = _evaluate_move

A more efficient Zobrist Hasher. For even more performance incremental hashing needs to be implemented

In [ ]:
import chess.polyglot


class ZobristHasher(chess.polyglot.ZobristHasher):

    def hash_castling(self, board: chess.Board) -> int:
        zobrist_hash = 0

        if board.castling_rights & chess.BB_H1:
            zobrist_hash ^= self.array[768]
        if board.castling_rights & chess.BB_A1:
            zobrist_hash ^= self.array[768 + 1]
        if board.castling_rights & chess.BB_H8:
            zobrist_hash ^= self.array[768 + 2]
        if board.castling_rights & chess.BB_A8:
            zobrist_hash ^= self.array[768 + 3]

        return zobrist_hash

    def hash_board(self, board: chess.BaseBoard) -> int:
        zobrist_hash = 0

        for square in chess.scan_reversed(board.occupied_co[chess.BLACK]):
            zobrist_hash ^= self.array[128 * board.piece_type_at(square) - 128 +
                                       square]

        for square in chess.scan_reversed(board.occupied_co[chess.WHITE]):
            zobrist_hash ^= self.array[128 * board.piece_type_at(square) - 64 +
                                       square]

        return zobrist_hash

    def set_piece(self, hash):
        hash ^= self.array[64 * 3 + 8 * 10]  # Sample zrobist xor
        hash ^= self.array[64 * 3 + 8 * 10]  # Sample zrobist xor
        hash ^= self.array[64 * 3 + 8 * 10]  # Sample zrobist xor
        hash ^= self.array[64 * 3 + 8 * 10]  # Sample zrobist xor


zobrist_hash_old = chess.polyglot.ZobristHasher(
    chess.polyglot.POLYGLOT_RANDOM_ARRAY
)
zobrist_hash_new = ZobristHasher(chess.polyglot.POLYGLOT_RANDOM_ARRAY)

board = chess.Board()

# move = list(board.legal_moves)[0]
# board.push(move)

%timeit zobrist_hash_old.hash_board(board)
%timeit zobrist_hash_new.hash_board(board)

hash = zobrist_hash_new.hash_board(board)

%timeit zobrist_hash_new.set_piece(hash)

In [ ]:
# Performance of board with at least one move does make a difference (as push() is precalculating some things)
move = list(board.legal_moves)[0]
board.push(move)

%timeit board._transposition_key()
%timeit board._transposition_key()
%timeit board._transposition_key()

board.pop()

Performance single move

In [ ]:
class Test(IterativeAlphaBetaCached):

    def __init__(self):
        self._value_counter = 0
        self._value2_counter = 0
        self._evaluate_counter = 0
        self._absolute_heuristic_counter = 0
        super().__init__()

    def _value(self, *args, **kwarg):
        self._value_counter += 1
        return super()._value(*args, **kwarg)

    def _value2(self, *args, **kwarg):
        self._value2_counter += 1
        return super()._value2(*args, **kwarg)

    def _evaluate(self, *args, **kwarg):
        self._evaluate_counter += 1
        return super()._evaluate(*args, **kwarg)

    def _absolute_heuristic(self, *args, **kwargs):
        self._absolute_heuristic_counter += 1
        return super()._absolute_heuristic(*args, **kwargs)

In [ ]:
class Board(chess.Board):

    def __init__(self, *args, **kwarg):
        self.push_counter = 0
        super().__init__(*args, **kwarg)

    def push(self, *args, **kwarg):
        self.push_counter += 1
        super().push(*args, **kwarg)

In [ ]:
engine = Test()
board = Board("rnbqkbnr/p2ppppp/8/1p6/3PP3/8/PP3PPP/RNBQKBNR b Kkq - 0 1")
%time result = engine._evaluate_move(board, ev2[0].move, 5)
print(engine._value_counter)
print(engine._value2_counter)
print(engine._evaluate_counter)
print(engine._absolute_heuristic_counter)
print(board.push_counter)

## Move Ordering 

## MDTF Search Algorithm

In [ ]:
import math


class IterativeMtdf(IterativeAlphaBetaCached):
    """Chess engine looking a fixed number of moves ahead using the alpha beta pruning algorithm"""

    def __init__(self, max_look_ahead_depth: int):
        super().__init__(max_look_ahead_depth)
        self.all_iterations = []

    def _mtdf(self, board: chess.Board, score: int, depth: int) -> int:
        upperbound = math.inf
        lowerbound = -math.inf

        iterations = 0

        while lowerbound < upperbound:
            beta = score + 1 if score == lowerbound else score
            score = self._value(board, depth, beta - 1, beta)
            if score < beta:
                upperbound = score
            else:
                lowerbound = score

            iterations += 1

        self.all_iterations.append(iterations)

        return score

    def _evaluate_move(
        self, board: chess.Board, scoredMove: ScoredMove, depth: int
    ):
        board.push(scoredMove.move)
        score = self._mtdf(board, scoredMove.score, depth)
        score *= self.PLAYER_MULTIPLIER[board.turn]
        board.pop()
        return ScoredMove(score=score, move=scoredMove.move)

    def _evaluate_moves(self, board: chess.Board):
        print(f"Max depth: {self.max_look_ahead_depth}")

        self.cache.clear()

        scoredMoves = [
            ScoredMove(score=0, move=move) for move in board.legal_moves
        ]

        for depth in range(self.max_look_ahead_depth + 1):
            scoredMoves = [
                self._evaluate_move(board, move, depth) for move in scoredMoves
            ]

            print(f"Depth {depth}")
            # print(f"result: {scoredMoves}\n")

        return scoredMoves

In [ ]:
import MinMaxTree

random.seed(42)

engine = IterativeMtdf(max_look_ahead_depth=2)
tree = MinMaxTree.add_tree_to_engine(engine, relative=True)
result_depth2_IterativeMtdf = engine.analyse(middlegame_board)

assert result_depth2_IterativeMtdf == result_iterativeAlphaBetaCached

In [ ]:
class IterativeMoveOrdered(IterativeAlphaBetaCached):

    def generateMoves(self, board: chess.Board, depth: int):
        pvMoves = []
        otherMoves = []

        for move in board.legal_moves:
            board.push(move)
            cacheKey = self.cache.get_key(board)
            try:
                type, value = self.cache.load_cache(cacheKey, depth - 1)

                if type == NodeType.EXACT:
                    pvMoves.append((value, move))
                else:
                    otherMoves.append(move)
            except:
                otherMoves.append(move)

            board.pop()

        pvMoves.sort(key=lambda tup: tup[0])
        return [move for _, move in pvMoves] + otherMoves

    @cache_alpha_beta
    def _value(
        self, board: chess.Board, depth: int, alpha: int, beta: int
    ) -> int:
        if score := self._evaluate(board):
            return score
        if depth == 0:
            return self._absolute_heuristic(board)

        for move in self.generateMoves(board, depth):
            board.push(move)
            value = -1 * self._value(board, depth - 1, -beta, -alpha)
            board.pop()

            if value >= beta:
                return value
            alpha = max(alpha, value)

        return alpha

In [ ]:
random.seed(42)

engine = IterativeMoveOrdered(max_look_ahead_depth=2)
# tree = MinMaxTree.add_tree_to_engine(engine, relative=True)
result_iterativeMoveOrdered = engine.analyse(middlegame_board)

assert result_iterativeMoveOrdered == result_iterativeAlphaBetaCached

In [ ]:
random.seed(42)

engine = IterativeAlphaBetaCached(max_look_ahead_depth=3)
tree = MinMaxTree.add_tree_to_engine(engine, relative=True)
engine.analyse(middlegame_board)
print(tree.count())

In [ ]:
random.seed(42)

engine = IterativeMoveOrdered(max_look_ahead_depth=3)
tree = MinMaxTree.add_tree_to_engine(engine, relative=True)
engine.analyse(middlegame_board)
print(tree.count())

In [ ]:
class PrincipalVariationSearch(IterativeMoveOrdered):

    @cache_alpha_beta
    def _value(
        self, board: chess.Board, depth: int, alpha: int, beta: int
    ) -> int:
        if score := self._evaluate(board):
            return score
        if depth == 0:
            return self._absolute_heuristic(board)

        for i, move in enumerate(self.generateMoves(board, depth)):
            board.push(move)

            if i == 0:
                value = -1 * self._value(board, depth - 1, -beta, -alpha)
            else:
                value = -1 * self._value(board, depth - 1, -alpha - 1, -alpha)
                if value > alpha:
                    value = -1 * self._value(board, depth - 1, -beta, -alpha)

            board.pop()

            if value >= beta:
                return value
            alpha = max(alpha, value)

        return alpha

In [ ]:
random.seed(42)

engine = PrincipalVariationSearch(max_look_ahead_depth=2)
# tree = MinMaxTree.add_tree_to_engine(engine, relative=True)
result_principalVariationSearch = engine.analyse(middlegame_board)

assert result_principalVariationSearch == result_iterativeAlphaBetaCached

In [ ]:
%load_ext line_profiler
random.seed(42)

engine = PrincipalVariationSearch(max_look_ahead_depth=3)
# tree = MinMaxTree.add_tree_to_engine(engine, relative=True)
%lprun -f engine._value result_principalVariationSearch = engine.analyse(middlegame_board)